<a href="https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_exportador_modelo_espa%C3%B1ol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="ffc800"> **Exportador de modelos de [Piper.](https://github.com/rhasspy/piper)**
## ![Piper logo](https://contribute.rhasspy.org/img/logo.png)
---
* Cuaderno creado por: [rmcpantoja](http://github.com/rmcpantoja)
* Decoración y traducción por: [Xx_Nessu_xX](http://github.com/XxNessuxX)

In [ ]:
#@markdown # <font color="ffc800"> **Instalar software.** 📦
#@markdown ---

print("\033[93mInstalando...")
!git clone -q https://github.com/rhasspy/piper
%cd /content/piper/src/python
!pip install -q cython>=0.29.0 espeak-phonemizer>=1.1.0 librosa>=0.9.2 numpy>=1.19.0 pytorch-lightning~=1.7.0 torch~=1.11.0
!pip install -q onnx onnxruntime-gpu
!bash build_monotonic_align.sh
!apt-get install espeak-ng
!pip install -q torchtext==0.12.0
# fixing recent compativility isswes:
!pip install -q torchaudio==0.11.0 torchmetrics==0.11.4
print("\033[93mHecho.")

In [ ]:
#@markdown # <font color="ffc800"> **Sección de generación de paquetes de voz.** 🗣️
#@markdown ---
%cd /content/piper/src/python
import os
import ipywidgets as widgets
from IPython.display import display
import json
from google.colab import output
guideurl = "https://github.com/rmcpantoja/piper/blob/master/notebooks/wav/es"
#@markdown #### *Descargar:*
#@markdown **ID de Drive o enlace de descarga directa del modelo en otra nube:**
model_id = "" # @param {type:"string"}
#@markdown **Id de Drive o enlace de descarga directa del archivo config.json:**
config_id = "" # @param {type:"string"}
#@markdown ---

#@markdown #### *Proceso de creación:*
#@markdown **Elige el código de idioma (formato iso639-1):**

#@markdown Puedes consultar la lista de códigos y nombres de idiomas [aquí](https://www.loc.gov/standards/iso639-2/php/English_list.php).

language = "es" #@param ["ca", "da", "de", "en", "en-us", "es", "fi", "fr", "grc", "is", "it", "k", "nb", "ne", "nl", "pl", "pt-br", "ru", "sv", "uk", "vi-vn-x-central", "yue"]
voice_name = "Mivoz" #@param {type:"string"}
voice_name = voice_name.lower()
quality = "medium" #@param ["high", "low", "medium", "x-low"]
def start_process():
    if not os.path.exists("/content/project/model.ckpt"):
        raise Exception("No se ha podido descargar el modelo. Asegúrate de que la compartición del archivo está puesto en público.")
    output.eval_js(f'new Audio("{guideurl}/starting.wav?raw=true").play()')
    !python -m piper_train.export_onnx "/content/project/model.ckpt" "{export_voice_path}/{export_voice_name}.onnx"
    print("\033[93mComprimiendo...")
    !tar -czvf "{packages_path}/voice-{export_voice_name}.tar.gz" -C "{export_voice_path}" .
    output.eval_js(f'new Audio("{guideurl}/success.wav?raw=true").play()')
    print("\033[93mHecho.")

export_voice_name = f"{language}-{voice_name}-{quality}"
export_voice_path = "/content/project/voice-"+export_voice_name
packages_path = "/content/project/packages"
if not os.path.exists(export_voice_path):
    os.makedirs(export_voice_path)
if not os.path.exists(packages_path):
    os.makedirs(packages_path)
print("\033[93mDescargando modelo y su configuración...")
if model_id.startswith("1"):
    !gdown -q "{model_id}" -O /content/project/model.ckpt
elif model_id.startswith("https://drive.google.com/file/d/"):
    !gdown -q "{model_id}" -O "/content/project/model.ckpt" --fuzzy
else:
    !wget "{model_id}" -O "/content/project/model.ckpt"
if config_id.startswith("1"):
    !gdown -q "{config_id}" -O "{export_voice_path}/{export_voice_name}.onnx.json"
elif config_id.startswith("https://drive.google.com/file/d/"):
    !gdown -q "{config_id}" -O "{export_voice_path}/{export_voice_name}.onnx.json" --fuzzy
else:
    !wget "{config_id}" -O "{export_voice_path}/{export_voice_name}.onnx.json"
#@markdown **¿Quieres escribir una tarjeta del modelo?** *(Opcional.)*
write_model_card = False #@param {type:"boolean"}
if write_model_card:
    with open(f"{export_voice_path}/{export_voice_name}.onnx.json", "r") as file:
        config = json.load(file)
    sample_rate = config["audio"]["sample_rate"]
    num_speakers = config["num_speakers"]
    output.eval_js(f'new Audio("{guideurl}/waiting.wav?raw=true").play()')
    text_area = widgets.Textarea(
        description = "Rellena la siguiente plantilla y pulsa Iniciar para generar el paquete de voz:",
        value=f'# Tarjeta de Modelo para {voice_name} ({quality})\n\n* Idioma: {language} (normalizado)\n* Hablantes: {num_speakers}\n* Calidad: {quality}\n* Frecuencia de muestreo: {sample_rate}Hz\n\n## Titular del modelo\n\n* Tu nick/nombre:\n\n## Dataset\n\n* URL: \n* Licencia: \n\n## Entrenamiento\n\nEntrenado de 0.\n\nAjustado de la voz: ',
        layout=widgets.Layout(width='500px', height='200px')
    )
    button = widgets.Button(description='Iniciar.')

    def create_model_card(button):
        model_card_text = text_area.value.strip()
        with open(f'{export_voice_path}/MODEL_CARD', 'w') as file:
            file.write(model_card_text)
        text_area.close()
        button.close()
        output.clear()
        start_process()

    button.on_click(create_model_card)

    display(text_area, button)
else:
    start_process()

In [ ]:
#@markdown # <font color="ffc800"> **Descarga/exporta tu paquete de voz generado.** 📥
#@markdown ---

#@markdown #### *¿Cómo quieres exportar tu modelo?*
export_mode = "Subirlo a mi Google Drive." #@param ["Descargar el paquete de voz en mi dispositivo. (Puede llevar algún tiempo.)", "Subirlo a mi Google Drive."]
print("\033[93mExportando paquete...")
if export_mode == "Descargar el paquete de voz en mi dispositivo. (Puede llevar algún tiempo.)":
    from google.colab import files
    files.download(f"{packages_path}/voice-{export_voice_name}.tar.gz")
    msg = "\033[93mEspera un momento mientras se descarga el paquete."
else:
    voicepacks_folder = "/content/drive/MyDrive/piper voice packages"
    from google.colab import drive
    drive.mount('/content/drive')
    if not os.path.exists(voicepacks_folder):
        os.makedirs(voicepacks_folder)
    !cp "{packages_path}/voice-{export_voice_name}.tar.gz" "{voicepacks_folder}"
    msg = f"\033[93mPuedes encontrar el paquete de voz generado en: {voicepacks_folder}."
print(f"\033[93mHecho. {msg}")

# "*¡Quiero probar este modelo! ¿Ya no necesito nada más?*"

No, ¡esto está casi terminado! Ahora puedes compartir tu paquete generado con tus amigos, subirlo a un almacenamiento en la nube y/o probarlo en:

* [El cuaderno de inferencia](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_inference_(ONNX).ipynb)
  * Ejecuta las celdas en orden para que funcione correctamente, al igual que con todos los cuadernos. Además, el cuaderno de inferencia te guiará a través del proceso utilizando la función de accesibilidad mejorada si lo deseas. Es fácil de usar. ¡Pruébalo!
* O a través del lector de pantalla NVDA.
  * Descarga e instala la última versión del [complemento](https://github.com/mush42/piper-nvda/releases).
  * Una vez que el complemento esté instalado, ve al menú NVDA/preferencias/opciones... y busca la categoría `Piper Voice Manager`.
  * Navega a través de los controles hasta que encuentres el botón `Install from local file`, presiona enter y selecciona el paquete generado en tus descargas.
  * Una vez que el paquete esté seleccionado e instalado, aplica los cambios y reinicia NVDA para actualizar la lista de voces.
* ¡Disfruta de tu creación!
